<a href="https://colab.research.google.com/github/Taedriel/ZSL-v2/blob/wordEmbeddingPython/zsl/word_embeddings/notebook/Hierarchical_clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install orange3 python-louvain networkx dendropy biopython scikit-bio --quiet --upgrade

In [ ]:
import numpy as np
import torch
import community.community_louvain as community
import dendropy
# if import error, launch import a second time, and it will be fine
from typing import Dict, Tuple, List, Callable
from tqdm import tqdm
from operator import attrgetter

from Orange.clustering.hierarchical import Tree, ClusterData, SingletonData
from Orange.data import Table, Domain
from Orange.data.variable import StringVariable

from Bio import Phylo
from io import StringIO
from sklearn.metrics.pairwise import cosine_similarity,cosine_distances

from itertools import chain
from collections import Counter, deque

from skbio import DistanceMatrix
from skbio.tree import nj
from sklearn.metrics import pairwise_distances



In [ ]:
class EmbeddingsLoader:

    """class that load an embeddings file to perform operation on it. Base class
     for multiple operations such as matrix similarity operations.
     """

    def __init__(self, filename : str):

        self.file = filename
        self.embeddings = {}

        self._load_file()

    def _load_file(self):
        try:
            with open(self.file, "r") as f:
                lines = f.readlines()
                
            for line in lines[1:]:
                data = line.split(",")
                self.embeddings[data[0]] = torch.FloatTensor(list(map(float, data[1:])))

        except IOError as e:
            raise IOError(f"No file {self.file}")

# New Solver

In [ ]:
GROUP_BY = "first superclass"
MYSTERY = "TOGUESS"
SIM_THRESOLD = 0.3

def left_join(complete_table, supp_info_table, key: str = "embeddings") -> Table:
    """add all <b> metas </b> column from supp_info_table to complete_table using key as joint
    """
    assert key in list(map(lambda x : x.name, supp_info_table.domain.metas)), "embeddings name not present in additional data"
    # assert len(complete_table) == len(supp_info_table), "table don't contain the same number of line"
    print(len(complete_table), len(supp_info_table))
    name_supp_data = [i.name for i in chain(supp_info_table.domain.metas, 
                                            supp_info_table.domain.variables, 
                                            supp_info_table.attributes) if i.name != key]
                                            
    supp_list_list = [[] for i in range(len(name_supp_data))]

    for s in complete_table:
        done = False
        for d in supp_info_table:
            if s[key] == d[key]:
                for i, name in enumerate(name_supp_data):
                    supp_list_list[i].append(d[name])
                done = True
                break
        if not done:
            for i, name in enumerate(name_supp_data):
                supp_list_list[i].append("?")

    for i, name in enumerate(name_supp_data):
        # print(f"adding {name}")
        complete_table = complete_table.add_column(StringVariable(name), supp_list_list[i])

    return complete_table

def parent_of_mystery(cluster):
    res = None
    for branch in cluster.branches:
        if branch.is_leaf:
            if branch.value.index == MYSTERY:
                return cluster
        else: 
            res = parent_of_mystery(branch)
            if res is not None:
                return res
    return None
    
def first_child(root):
    if root.is_leaf:
        return root
    else:
        return first_child(root.branches[0])


def closest_to(cluster, mystery_index):
    if len(cluster.branches) == 1:
        return None

    next = False
    for i, branch in enumerate(cluster.branches):
        if next:
            return first_child(branch)

        if branch.is_leaf:
            if branch.value.index == mystery_index:
                if i == 0:
                    next = True
                else:
                    return first_child(cluster.branches[i-1])

def add_to_list(cluster, list_to_add_to):
    """ decompose a cluster tree by adding the index of all children in the list
    """
    if cluster.is_leaf:
        list_to_add_to.append(cluster.value.index)

    for i, branch in enumerate(cluster.branches):
        add_to_list(branch, list_to_add_to)

def compute(lst, tips):
    # return max(lst,key=lst.count)
    weighted_lst = {elem: 0 for elem in set(lst)}
    for elem in lst:
        if elem in tips:
            weighted_lst[elem] += 3
        else:
            weighted_lst[elem] += 1

    most_common = sorted([(key, elem) for key, elem in weighted_lst.items()], key = lambda x : x[1], reverse = True)
    return most_common

def Orange_tree_to_newick(root, depth = [0]):

    if root.is_leaf:
        return str(root.value.index)  +  ":" + f"{root.value.height - sum(depth):.6f}"

    concat = "("
    branch_length = float(root.value.height - sum(depth))
    for branch in root.branches:
        concat += Orange_tree_to_newick(branch, [*depth, branch_length]) + ", "

    concat = concat[:-2] + ")" +  ":" + f"{branch_length:.6f}"
    return concat

def biotree_to_Orange_tree(tree):

    def recur_parse(root, acc, depth):
        if root.is_terminal():
            val = root.name
            leaf = Tree(SingletonData(range = range(len(acc), len(acc)+1), 
                                 height = depth + (root.branch_length or 0), 
                                 index = val), ())
            acc.append(leaf)
            return leaf

        current_depth = depth + (root.branch_length or 0)
        list_cla = []
        for cla in root:
            sub_tree = recur_parse(cla, acc, current_depth)
            list_cla.append(sub_tree)
        node = Tree(ClusterData(range = range(list_cla[0]._Tree__value.range.start, list_cla[-1]._Tree__value.range.stop),
                            height = current_depth), tuple(list_cla))
        return node

    orange_tree = recur_parse(tree.root, [], 0)
    return orange_tree

def reroot_tree(tree):
    newick_tree = Orange_tree_to_newick(tree)
    biotree = Phylo.read(StringIO(newick_tree), "newick")
    biotree.root_with_outgroup(biotree.root.find_clades("NULL"))
    return biotree_to_Orange_tree(biotree)

def njt(table, key : str):

    embeddings = {}
    for line in table:
        embeddings[str(line[key].value)] = list(line.attributes()) 
        
    ids = list(map(lambda x : x.replace(" ", "_").replace("-", "_").replace("'", "_"), embeddings.keys()))
    data = np.array([item for item in embeddings.values()])
    
    cos_A = pairwise_distances(data, metric="cosine")
    
    def constructor(x):
        biotree = Phylo.read( StringIO(x), "newick")
        return biotree_to_Orange_tree(biotree)

    dm = DistanceMatrix(cos_A, ids)
    tree = nj(dm, result_constructor = constructor)

    return tree

def preorder(tree, branches=attrgetter("branches")):
    stack = deque([tree])
    while stack:
        current = stack.popleft()
        yield current
        children = branches(current)
        if children:
            stack.extendleft(reversed(children))

def clusters_at_height(root, height):
    """Return a list of clusters by cutting the clustering at `height`.
    """
    lower = set()
    cluster_list = []
    for cl in preorder(root):
        if cl in lower:
            continue
        if cl.value.height >= height:
            cluster_list.append(cl)
            lower.update(preorder(cl))
    return cluster_list



def clusterize(table : Table, threshold, key = "embeddings") -> Table:
    """clusterize a Oranga Table based on the height of THRESOLD
    """

    root = njt(table, key)
    # root = reroot_tree(root)

    parent_cluster = parent_of_mystery(root)
    if threshold is None:
        threshold = parent_cluster.value.height - 0.001

    cluster_tree = clusters_at_height(root, threshold)

    list_cluster = {}
    closest = None
    mystery_len_cluster = -1
    for i, cluster in enumerate(cluster_tree):
        cluster_name = 'C' + str(i) 

        current = []
        add_to_list(cluster, current)
        if MYSTERY in current: 
            mystery_len_cluster = len(current)
            closest = closest_to(parent_cluster, MYSTERY)

        for item in current:
            list_cluster[item] = cluster_name
        # print(cluster_name, list(map(lambda x: table[x]["embeddings"].value, current)))

    list_vocab = [line[key].value.replace(" ", "_").replace("-", "_").replace("'", "_") for line in table]
    table = table.add_column(StringVariable("Cluster"), [list_cluster[i] if i in list_cluster.keys() else "Out" for i in list_vocab])

    return table, closest.value.index if closest is not None else None, threshold, len(cluster_tree)

from typing import List
def one_pass(table, toguess_table, keep_cluster_line : bool = False, cluster_thresold : float = CLUSTER_THRESOLD, sim_thresold : float = SIM_THRESOLD, tips : List[str] = []):
    assert GROUP_BY in list(map(lambda x: x.name, chain(table.domain.metas, 
                                                        table.domain.variables, 
                                                        table.domain.attributes))), "Group by not in the Table !"
    supp_data = {
        "sim_thresold"       : sim_thresold,
        "keep_cluster_line"  : keep_cluster_line,
    }

    supp_data["format_at_beginning"] = (len(table), "x", len(table.domain.attributes))
    if len(table) < 3:
        return [], supp_data
    table, closest, thresold, nb_cluster = clusterize(table, cluster_thresold)
    supp_data["cluster_thresold"]    = thresold
    supp_data["closest_to_myster"]   = closest # table[]["embeddings"].value if closest is not True else None,
    supp_data["number_of_cluster"]   = nb_cluster
    #===========================================================================
    # Cluster split
    toguess_cluster = [d["Cluster"] for d in table if d["embeddings"] == MYSTERY][0]

    in_cluster_table  = Table.from_list(table.domain, [d for d in table if d["Cluster"].value == toguess_cluster])
    out_cluster_table = Table.from_list(table.domain, [d for d in table if d["Cluster"].value != toguess_cluster])
    #===========================================================================
    # Group by computation
    supp_data["cluster_size"] = len(in_cluster_table)
    if len(in_cluster_table) <= 1: return [], supp_data
    
    main_superclass_count_list = compute([row[GROUP_BY].value for row in in_cluster_table], tips)
    #equality case with "?", take the second
    ind = 1 if main_superclass_count_list[0][0] == "?" and len(main_superclass_count_list) > 1 else 0
    main_superclass = main_superclass_count_list[ind][0]
    supp_data["cluster_name"] = main_superclass

    nb_dimension = len(list(in_cluster_table.domain.attributes))
    average_cluster = Table.from_list(in_cluster_table.domain, [
        [sum([line[i] for line in in_cluster_table]) / nb_dimension  for i in in_cluster_table.domain.attributes] + ["cluster_average"]
    ])

    # main_superclass_table = Table.from_list(superclass_embeddings.domain, [i for i in superclass_embeddings if i["embeddings"] == main_superclass])
    main_superclass_table = Table.concatenate([in_cluster_table, Table.from_table(in_cluster_table.domain, average_cluster)])
    #===========================================================================
    # thresold computation
    to_copy_row_instance = [d for d in main_superclass_table if d["embeddings"] == MYSTERY][0]
    to_copy = list(to_copy_row_instance.attributes())

    to_compare_row_instance = [d for d in main_superclass_table if d["Cluster"] == "?"][0]
    to_compare = list(to_compare_row_instance.attributes())

    dead_row = [k for k, (i, j) in enumerate(zip(to_copy, to_compare)) if abs(i - j) <= sim_thresold]
    supp_data["removed_col"] = len(dead_row) 
    #===========================================================================
    # reconstruct the table filtering dead row and cluster. Remove used cluster row if 
    # keep_cluster_line is set to False
    new_domain = Domain(attributes = [i for i in out_cluster_table.domain.attributes if int(i.name) not in dead_row], 
                        metas      = [i for i in out_cluster_table.domain.metas if i.name != "Cluster"])

    # do the same on the data
    data_attr, data_meta = [], []
    whole_data = list(out_cluster_table) + list(toguess_table)
    if keep_cluster_line: whole_data += list(in_cluster_table)

    for rowinstance in whole_data:
        data_attr.append([rowinstance[k] for k, i in enumerate(out_cluster_table.domain.attributes) if int(i.name) not in dead_row])
        # data_attr.append([rowinstance[k] for k, i in enumerate(out_cluster_table.domain.attributes)])
        data_meta.append([rowinstance.metas[k] for k, i in enumerate(out_cluster_table.domain.metas) if i.name != "Cluster"])


    return Table.from_numpy(new_domain, X = data_attr, metas = data_meta), supp_data


def standardize_first(table):
    values = table[0]
    mean = np.mean(values)
    std  = np.std(values)

    for v in range(len(values)):
        values[v] = (values[v] - mean) / std

    return Table.from_numpy(table.domain, [values], None, table.metas)

In [ ]:
def solve_mystery(complete_table, mystery, cluster_thresold_lambda, sim_thresold_lambda, tips):

    toguess_table = Table.from_numpy(complete_table.domain, [np.array(mystery)], Y = None, metas = np.char.asarray([[MYSTERY, "?", "?"]]))
    # toguess_table = standardize_first(toguess_table)

    null_table = Table.from_numpy(complete_table.domain, [np.array([1 for i in range(len(complete_table.domain.attributes))])], Y = None, metas = np.char.asarray([["NULL", "?", "?"]]))

    table = Table.concatenate([complete_table, toguess_table, null_table])
    old_table = table

    advancement = []
    for i in range(5):
        old_table = table
        table, data = one_pass(table, toguess_table,
                                      keep_cluster_line = False, 
                                      cluster_thresold  = cluster_thresold_lambda(i), 
                                      sim_thresold      = sim_thresold_lambda(i),
                                      tips = tips)
        advancement.append(data)

        if len(table) <= 1 or data["cluster_size"] < 10:
            break
    return advancement

In [ ]:
generic_table = Table("/content/ResNet50-average.csv")
supp_info_table = Table("/content/class_map_imagenet.csv")
generic_table = left_join(generic_table, supp_info_table)

print(len(generic_table))

# superclass_embeddings = Table("/content/custom-wikipedia2vec-300_superclass.csv")

996 999
996


In [ ]:
myster_file = EmbeddingsLoader("/content/mystery_CNN.csv")

In [ ]:
def format_result(list_dict):
    superclass_list = []
    for dic in list_dict:
        if type(dic) == type(dict()) and "cluster_name" in dic.keys():
            superclass_list.append(f"{dic['cluster_name']}[{round(dic['cluster_size'] / len(generic_table) * 100, 1)}%]({dic['closest_to_myster']})")
    
    return superclass_list

# cluster_thresold_lambda = lambda x : 0.10 + 0.1 * x
cluster_thresold_lambda     = lambda x : None
sim_thresold_lambda     = lambda x : 0.3

tips = [["bear"],
        ["bear"],
        ["bear"],
        ["monotreme"],
        ["monotreme"],
        ["monotreme"],
        ["cat"],
        ["rodent"],
        ["dog"],
        ["bear"],
        ["bear"],
        ["bear"],
        ["bear"],
        ["bear"],
        ["bear"]]

for k, (i, embeddings) in enumerate(myster_file.embeddings.items()):

    result = solve_mystery(generic_table, embeddings, cluster_thresold_lambda, sim_thresold_lambda, tips[k])
    print(f"{' + '.join(format_result(result)): <80}\t\t{result}")


vehicle[0.8%](jinrikisha)                                                       		[{'sim_thresold': 0.3, 'keep_cluster_line': False, 'format_at_beginning': (998, 'x', 2048), 'cluster_thresold': 0.086112, 'closest_to_myster': 'jinrikisha', 'number_of_cluster': 205, 'cluster_size': 8, 'cluster_name': 'vehicle', 'removed_col': 960}]
dog[1.7%](schipperke) + dog[11.5%](Cardigan) + bird[15.5%](crane) + bug[15.7%](rapeseed) + ?[0.2%](NULL)		[{'sim_thresold': 0.3, 'keep_cluster_line': False, 'format_at_beginning': (998, 'x', 2048), 'cluster_thresold': 0.122675, 'closest_to_myster': 'schipperke', 'number_of_cluster': 299, 'cluster_size': 17, 'cluster_name': 'dog', 'removed_col': 1145}, {'sim_thresold': 0.3, 'keep_cluster_line': False, 'format_at_beginning': (982, 'x', 903), 'cluster_thresold': 0.048677, 'closest_to_myster': 'Cardigan', 'number_of_cluster': 88, 'cluster_size': 115, 'cluster_name': 'dog', 'removed_col': 109}, {'sim_thresold': 0.3, 'keep_cluster_line': False, 'format_at_beginning'